In [1]:
import numpy as np
import pandas as pd
from starfish.plot import *

In [2]:
from starfish.set import settings
settings.var_name.remove('N_eff')
settings.cosmo_num-=1

In [3]:
def var_filter(fisher,lcdm,neff_fixed):
    df=fisher.copy()
    if lcdm:
        df=df.drop('w0',axis=0)
        df=df.drop('w0',axis=1)
        df=df.drop('wa',axis=0)
        df=df.drop('wa',axis=1)
    if neff_fixed:
        df=df.drop('N_eff',axis=0)
        df=df.drop('N_eff',axis=1)

    return df

# nuisance parameters of different tracers/surveys should be independent of each other

In [4]:
DESI_multi=var_filter(pd.read_excel('../data/DESI_all/valid.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=True)
cov=pd.DataFrame(np.linalg.inv(DESI_multi.values),index=DESI_multi.index,columns=DESI_multi.columns)
DESI_multi_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

In [5]:
JPAS_multi=var_filter(pd.read_excel('../data/jpas_all/valid.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=True)
cov=pd.DataFrame(np.linalg.inv(JPAS_multi.values),index=JPAS_multi.index,columns=JPAS_multi.columns)
JPAS_multi_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

In [6]:
PFS_ELG=var_filter(pd.read_excel('../data/PFS/fisher.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=True)
cov=pd.DataFrame(np.linalg.inv(PFS_ELG.values),index=PFS_ELG.index,columns=PFS_ELG.columns)
PFS_ELG_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

# CMB-S4 + LiteBIRD + DESI multitracer

In [7]:
cmb_fisher=var_filter(pd.read_excel('../data/CMB/all.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=True)
cmb_fisher

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,tau
omega_m0,1.851288e+08,-6.928981e+08,1.371057e+08,-4.740936e+06,-5.782434e+07,-1.244715e+07,4.485034e+07
omega_b0,-6.928981e+08,3.023402e+09,-4.583542e+08,3.206098e+07,2.226094e+08,4.714503e+07,-1.714219e+08
h,1.371057e+08,-4.583542e+08,1.125702e+08,-1.894718e+06,-3.817385e+07,-7.840066e+06,2.962660e+07
n_s,-4.740936e+06,3.206098e+07,-1.894718e+06,1.046895e+06,1.665063e+06,3.149959e+05,-1.284039e+06
sigma_8,-5.782434e+07,2.226094e+08,-3.817385e+07,1.665063e+06,2.236167e+07,5.252308e+06,-1.731189e+07
m_nu,-1.244715e+07,4.714503e+07,-7.840066e+06,3.149959e+05,5.252308e+06,1.282975e+06,-4.090356e+06
tau,4.485034e+07,-1.714219e+08,2.962660e+07,-1.284039e+06,-1.731189e+07,-4.090356e+06,1.374361e+07


In [8]:
cmb_cov=pd.DataFrame(np.linalg.inv(cmb_fisher.values),index=cmb_fisher.index,columns=cmb_fisher.columns)
cmb_cov_marg=cmb_cov.drop('tau',axis=0)
cmb_cov_marg=cmb_cov_marg.drop('tau',axis=1)
cmb_fisher_marg=pd.DataFrame(np.linalg.inv(cmb_cov_marg),index=settings.var_name,columns=settings.var_name)
cmb_fisher_marg

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu
omega_m0,3.876607e+07,-1.334868e+08,4.042346e+07,-5.506558e+05,-1.329430e+06,9.011528e+05
omega_b0,-1.334868e+08,8.852823e+08,-8.882606e+07,1.604536e+07,6.680925e+06,-3.873349e+06
h,4.042346e+07,-8.882606e+07,4.870520e+07,8.732384e+05,-8.552429e+05,9.773665e+05
n_s,-5.506558e+05,1.604536e+07,8.732384e+05,9.269296e+05,4.764654e+04,-6.715809e+04
sigma_8,-1.329430e+06,6.680925e+06,-8.552429e+05,4.764654e+04,5.550676e+05,9.996606e+04
m_nu,9.011528e+05,-3.873349e+06,9.773665e+05,-6.715809e+04,9.996606e+04,6.560846e+04


In [9]:
fisher1_combined=DESI_multi_valid+cmb_fisher_marg
fisher1_combined.to_excel('../data/combined/cmb+dmul-lcdm-fixed.xlsx')
cov1_combined=pd.DataFrame(np.linalg.inv(fisher1_combined),index=fisher1_combined.index,columns=fisher1_combined.columns)
cov1_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu
omega_m0,3.183983e-06,3.475430e-07,-2.349275e-06,-7.220722e-07,-2.877698e-06,0.000016
omega_b0,3.475430e-07,4.127347e-08,-2.566042e-07,-1.101656e-07,-3.818162e-07,0.000002
h,-2.349275e-06,-2.566042e-07,1.802413e-06,3.198843e-07,2.503132e-06,-0.000014
n_s,-7.220722e-07,-1.101656e-07,3.198843e-07,2.281219e-06,-1.699285e-07,0.000001
sigma_8,-2.877698e-06,-3.818162e-07,2.503132e-06,-1.699285e-07,7.837225e-06,-0.000033
m_nu,1.578402e-05,1.991772e-06,-1.350439e-05,1.377160e-06,-3.301248e-05,0.000171


In [10]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 1.8e-03
-- omega_b0: 2.0e-04
--        h: 1.3e-03
--      n_s: 1.5e-03
--  sigma_8: 2.8e-03
--     m_nu: 1.3e-02


# CMB-S4 + LiteBIRD + DESI multitracer

In [11]:
fisher2=pd.DataFrame(JPAS_multi_valid+PFS_ELG_valid,index=settings.var_name,columns=settings.var_name)
fisher2.to_excel('../data/combined/pelg+jmul-lcdm-fixed.xlsx')

In [12]:
fisher2_combined=JPAS_multi_valid+PFS_ELG_valid+cmb_fisher_marg
fisher2_combined.to_excel('../data/combined/cmb+pelg+jmul-lcdm-fixed.xlsx')
cov2_combined=pd.DataFrame(np.linalg.inv(fisher2_combined),index=fisher2_combined.index,columns=fisher2_combined.columns)
cov2_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu
omega_m0,4.327486e-06,5.008204e-07,-3.263318e-06,-1.182106e-06,-5.271530e-06,2.609831e-05
omega_b0,5.008204e-07,6.173239e-08,-3.804124e-07,-1.584457e-07,-7.180440e-07,3.419774e-06
h,-3.263318e-06,-3.804124e-07,2.538984e-06,6.665783e-07,4.524369e-06,-2.209051e-05
n_s,-1.182106e-06,-1.584457e-07,6.665783e-07,2.325928e-06,2.197480e-07,-8.807956e-07
sigma_8,-5.271530e-06,-7.180440e-07,4.524369e-06,2.197480e-07,1.463124e-05,-6.016785e-05
m_nu,2.609831e-05,3.419774e-06,-2.209051e-05,-8.807956e-07,-6.016785e-05,2.817578e-04


In [13]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov2_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 2.1e-03
-- omega_b0: 2.5e-04
--        h: 1.6e-03
--      n_s: 1.5e-03
--  sigma_8: 3.8e-03
--     m_nu: 1.7e-02
